### Постановка задачи

Необходимо предсказать биологический ответ молекул (столбец 'Activity') по их химическому составу (столбцы D1-D1776).

Данные представлены в формате CSV. Каждая строка представляет молекулу.

Первый столбец Activity содержит экспериментальные данные, описывающие фактический биологический ответ [0, 1];
Остальные столбцы D1-D1776 представляют собой молекулярные дескрипторы — это вычисляемые свойства, которые могут фиксировать некоторые характеристики молекулы, например размер, форму или состав элементов.
Предварительная обработка не требуется, данные уже закодированы и нормализованы.

В качестве метрики будем использовать F1-score.

Необходимо обучить две модели: логистическую регрессию и случайный лес. Далее нужно сделать подбор гиперпараметров с помощью базовых и продвинутых методов оптимизации. Важно использовать все четыре метода (GridSeachCV, RandomizedSearchCV, Hyperopt, Optuna) хотя бы по разу, максимальное количество итераций не должно превышать 50.

In [1]:
#импорт библиотек
import numpy as np #для матричных вычислений
import pandas as pd #для анализа и предобработки данных
import matplotlib.pyplot as plt #для визуализации
import seaborn as sns #для визуализации

import warnings
warnings.filterwarnings("ignore")

from sklearn import linear_model #линейные модели
from sklearn import tree #деревья решений
from sklearn import ensemble #ансамбли
from sklearn import metrics #метрики
from sklearn import preprocessing #предобработка
from sklearn.model_selection import train_test_split #сплитование выборки
from sklearn.model_selection import cross_val_score #Кросс валидация

# Импорт оптимизаторов параметров
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV

import hyperopt
from hyperopt import hp, fmin, tpe, Trials

import optuna

%matplotlib inline
plt.style.use('seaborn')

In [2]:
data = pd.read_csv('data/_train_sem09.csv')
data.head()

,Activity,D1,D2,D3,D4,D5,D6,D7,D8,D9,...,D1767,D1768,D1769,D1770,D1771,D1772,D1773,D1774,D1775,D1776
0,1,0.000000,0.497009,0.10,0.0,0.132956,0.678031,0.273166,0.585445,0.743663,...,0,0,0,0,0,0,0,0,0,0
1,1,0.366667,0.606291,0.05,0.0,0.111209,0.803455,0.106105,0.411754,0.836582,...,1,1,1,1,0,1,0,0,1,0
2,1,0.033300,0.480124,0.00,0.0,0.209791,0.610350,0.356453,0.517720,0.679051,...,0,0,0,0,0,0,0,0,0,0
3,1,0.000000,0.538825,0.00,0.5,0.196344,0.724230,0.235606,0.288764,0.805110,...,0,0,0,0,0,0,0,0,0,0
4,0,0.100000,0.517794,0.00,0.0,0.494734,0.781422,0.154361,0.303809,0.812646,...,0,0,0,0,0,0,0,0,0,0


In [3]:
X = data.drop(['Activity'], axis=1)
y = data['Activity']

In [4]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 42, test_size = 0.3)

# Мое решение

Сначала получим значения меток F1-score для логистической регресии и случайного леса без использования методов оптимизации, чтобы иметь в дальнейшем базу для сравнения.

Для обобщения результатов создаем два словаря logreg и ranfor, в которые будем заносить результаты обучения каждым методом. 

In [5]:
logreg = {}     # словарь для методов логистической регрессии
ranfor = {}     # словарь для методов случайного леса 

### Логистическая регрессия (без методов оптимизации)

In [6]:
#Создаем объект класса логистическая регрессия
log_reg = linear_model.LogisticRegression(random_state=42, max_iter = 50)
#Обучаем модель
log_reg.fit(X_train, y_train)

y_test_pred = log_reg.predict(X_test)
print('f1_score на тестовом наборе: {:.2f}'.format(metrics.f1_score(y_test, y_test_pred)))

f1_score на тестовом наборе: 0.78


In [7]:
# Записываем получившееся значение в словарь для дальнейшего сравнения
logreg['LogReg_init'] = round(metrics.f1_score(y_test, y_test_pred), 3)

### Случайный лес (без методов оптимизации)

In [8]:
#Создаем объект класса случайный лес
rf = ensemble.RandomForestClassifier(random_state=42)
#Обучаем модель
rf.fit(X_train, y_train)

y_test_pred = rf.predict(X_test)
print('f1_score на тестовом наборе: {:.2f}'.format(metrics.f1_score(y_test, y_test_pred)))

f1_score на тестовом наборе: 0.82


In [9]:
# Записываем получившееся значение в словарь для дальнейшего сравнения
ranfor['RanFor_init'] = round(metrics.f1_score(y_test, y_test_pred), 3)

Далее последовательно используются методы оптимизации

# Метод GridSeachCV

### Логистическая регрессия, метод GridSeachCV

In [10]:
param_grid = [
    {'penalty' : ['l2', 'none'], # тип регуляризации
    'solver' : ['newton-cg', 'lbfgs', 'sag'], # алгоритм оптимизации
    'C' : [0.01, 0.1, 0.3, 0.5, 0.7, 0.9, 1]}, # уровень силы регурялизации
    # другой набор гиперпараметров
    {'penalty': ['l1', 'l2'] ,
    'solver': ['liblinear', 'saga'],
    'C': [0.01, 0.1, 0.3, 0.5, 0.7, 0.9, 1]}
]

grid_search_LR = GridSearchCV(
    estimator = linear_model.LogisticRegression(random_state= 42, max_iter = 50),
    param_grid = param_grid,
    cv = 5,
    n_jobs = -1,
    scoring='f1'
)
%time grid_search_LR.fit(X_train, y_train)

y_test_pred = grid_search_LR.predict(X_test)

print('f1_score на тестовом наборе: {:.2f}'.format(metrics.f1_score(y_test, y_test_pred)))
print("Наилучшие значения гиперпараметров: {}".format(grid_search_LR.best_params_))

Wall time: 9min 1s
f1_score на тестовом наборе: 0.79
Наилучшие значения гиперпараметров: {'C': 0.1, 'penalty': 'l2', 'solver': 'saga'}


In [11]:
# Записываем получившееся значение в словарь для дальнейшего сравнения
logreg['LogReg_GridSeach'] = round(metrics.f1_score(y_test, y_test_pred), 3)

### Случайный лес, метод GridSeachCV

In [12]:
param_distributions = {'n_estimators': list(range(80, 200, 30)),
              'min_samples_leaf': list(np.linspace(5, 25, 5, dtype=int)),
              'max_depth': list(np.linspace(1, 40, 5, dtype=int))
              }
            
random_search_forest = RandomizedSearchCV(
    estimator=ensemble.RandomForestClassifier(random_state=42), 
    param_distributions=param_distributions, 
    cv=5,
    n_iter = 50, 
    n_jobs = -1,
    scoring = 'f1'
)  
%time random_search_forest.fit(X_train, y_train) 

y_test_pred = random_search_forest.predict(X_test)
print('f1_score на тестовом наборе: {:.2f}'.format(metrics.f1_score(y_test, y_test_pred)))
print("Наилучшие значения гиперпараметров: {}".format(random_search_forest.best_params_))

Wall time: 59.4 s
f1_score на тестовом наборе: 0.82
Наилучшие значения гиперпараметров: {'n_estimators': 110, 'min_samples_leaf': 5, 'max_depth': 40}


In [13]:
# Записываем получившееся значение в словарь для дальнейшего сравнения
ranfor['RanFor_GridSeach'] = round(metrics.f1_score(y_test, y_test_pred), 3)

# Метод RandomizedSeachCV

### Логистическая регрессия, метод RandomizedSeachCV

In [14]:
param_distributions = [
                        {'penalty': ['l2', 'none'] , # тип регуляризации
                         'solver' : ['newton-cg', 'lbfgs', 'sag'], # алгоритм оптимизации
                          'C': list(np.linspace(0.01, 1, 10, dtype=float))
                        },
                        {'penalty': ['l1', 'l2'] ,
                         'solver': ['liblinear', 'saga'],
                          'C': list(np.linspace(0.01, 1, 10, dtype=float))
                        }]
random_search_LR = RandomizedSearchCV(
    estimator=linear_model.LogisticRegression(random_state=42, max_iter= 50), 
    param_distributions=param_distributions, 
    cv=5,
    n_iter = 20, 
    n_jobs = -1,
    scoring='f1'
)  
%time random_search_LR.fit(X_train, y_train) 

y_test_pred = random_search_LR.predict(X_test)
print('f1_score на тестовом наборе: {:.2f}'.format(metrics.f1_score(y_test, y_test_pred)))
print("Наилучшие значения гиперпараметров: {}".format(random_search_LR.best_params_))

Wall time: 4min 37s
f1_score на тестовом наборе: 0.79
Наилучшие значения гиперпараметров: {'solver': 'saga', 'penalty': 'l2', 'C': 0.12}


In [15]:
# Записываем получившееся значение в словарь для дальнейшего сравнения
logreg['LogReg_RandSeach'] = round(metrics.f1_score(y_test, y_test_pred), 3)

### Случайный лес, метод RandomizedSeachCV

In [16]:
param_distributions = {'n_estimators': list(range(80, 200, 30)),
              'min_samples_leaf': list(np.linspace(5, 25, 5, dtype=int)),
              'max_depth': list(np.linspace(1, 40, 5, dtype=int))
              }
            
random_search_RF = RandomizedSearchCV(
    estimator=ensemble.RandomForestClassifier(random_state=42), 
    param_distributions=param_distributions, 
    cv=5,
    n_iter = 50, 
    n_jobs = -1,
    scoring= 'f1'
)  
%time random_search_RF.fit(X_train, y_train) 

y_test_pred = random_search_RF.predict(X_test)
print('f1_score на тестовом наборе: {:.2f}'.format(metrics.f1_score(y_test, y_test_pred)))
print("Наилучшие значения гиперпараметров: {}".format(random_search_RF.best_params_))

Wall time: 1min 14s
f1_score на тестовом наборе: 0.82
Наилучшие значения гиперпараметров: {'n_estimators': 170, 'min_samples_leaf': 5, 'max_depth': 30}


In [17]:
# Записываем получившееся значение в словарь для дальнейшего сравнения
ranfor['RanFor_RandSeach'] = round(metrics.f1_score(y_test, y_test_pred), 3)

# Метод Hyperopt

In [18]:
# Уточняем текущую версию HyperOpt
print("Версия Hyperopt : {}".format(hyperopt.__version__))

Версия Hyperopt : 0.2.5


### Логистическая регрессия, метод HyperOpt

In [19]:
space ={
    'penalty' : hp.choice(label='penalty', options= ['l1', 'l2']), # тип регуляризации
    'solver' : hp.choice(label = 'solver', options= ['liblinear', 'saga']), # алгоритм оптимизации
    'C' : hp.loguniform(label='C', low=-2*np.log(10), high=2*np.log(10)), # уровень силы регурялизации
    'max_iter': hp.choice('max_iter', [50])}

In [20]:
random_state = 42
def hyperopt_logr(params, cv=5, X=X_train, y=y_train, random_state=random_state):
    params = {'penalty': params['penalty'],
              'solver': params['solver'],
              'C': params['C'],
              'max_iter': params['max_iter']
              }
    model_HO = linear_model.LogisticRegression(**params, random_state=random_state)

    # обучаем модель
    model_HO.fit(X, y)
    score = cross_val_score(model_HO, X, y, cv=cv, scoring="f1", n_jobs=-1).mean()


    # метрику необходимо минимизировать, поэтому ставим знак минус
    return -score

In [21]:
%%time
# начинаем подбор гиперпараметров
trials = Trials() # используется для логирования результатов

best=fmin(hyperopt_logr, # наша функция 
          space=space, # пространство гиперпараметров
          algo=tpe.suggest, # алгоритм оптимизации, установлен по умолчанию, задавать необязательно
          max_evals=50, # максимальное количество итераций
          trials=trials, # логирование результатов
          # rstate = np.random.default_rng(42) # для версии 0.2.7
          rstate=np.random.RandomState(42) # (random_state)# фиксируем для повторяемости результата
         )
print("Наилучшие значения гиперпараметров {}".format(best))

100%|████████████████████████████████████████████████| 50/50 [07:58<00:00,  9.57s/trial, best loss: -0.777617316199278]
Наилучшие значения гиперпараметров {'C': 0.7038661632470573, 'max_iter': 0, 'penalty': 0, 'solver': 1}
Wall time: 7min 58s


Поскольку данная версия hyperopt в качестве индексов массива выдает для логистической регрессии только индексы массива, то воспользуемся space_eval, чтобы обращаться к элементам массива напрямую

In [22]:
from hyperopt import space_eval
hyperparams = space_eval(space, best)
hyperparams

{'C': 0.7038661632470573, 'max_iter': 50, 'penalty': 'l1', 'solver': 'saga'}

In [23]:
# рассчитаем точность для тестовой выборки
model_HO = linear_model.LogisticRegression(
    random_state=random_state,
    penalty = hyperparams['penalty'],
    solver = hyperparams['solver'],
    C = hyperparams['C'],
    max_iter = hyperparams['max_iter']
    )
model_HO.fit(X_train, y_train)

y_test_pred = model_HO.predict(X_test)
print('f1_score на тестовом наборе: {:.2f}'.format(metrics.f1_score(y_test, y_test_pred)))

f1_score на тестовом наборе: 0.80


In [24]:
# Записываем получившееся значение в словарь для дальнейшего сравнения
logreg['LogReg_HO'] = round(metrics.f1_score(y_test, y_test_pred), 3)

### Случайный лес, метод HyperOpt

In [25]:
# зададим пространство поиска гиперпараметров
space={'n_estimators': hp.quniform('n_estimators', 80, 200, 1),
       'max_depth' : hp.quniform('max_depth', 1, 30, 1),
       'min_samples_leaf': hp.quniform('min_samples_leaf', 5, 25, 1)
      }

In [26]:
random_state = 42
def hyperopt_rf(params, cv=5, X=X_train, y=y_train, random_state=random_state):
    
    params = {'n_estimators': int(params['n_estimators']), 
              'max_depth': int(params['max_depth']), 
             'min_samples_leaf': int(params['min_samples_leaf'])
    }
    # используем эту комбинацию для построения модели   
    model_HO_RF = ensemble.RandomForestClassifier(**params, random_state=random_state)

    # обучаем модель
    model_HO_RF.fit(X, y)
    score = cross_val_score(model_HO_RF, X, y, cv=cv, scoring="f1", n_jobs=-1).mean()

    return -score  

In [27]:
%%time
# начинаем подбор гиперпараметров
trials = Trials() # используется для логирования результатов

best=fmin(hyperopt_rf, # наша функция 
          space=space, # пространство гиперпараметров
          algo=tpe.suggest, # алгоритм оптимизации, установлен по умолчанию, задавать необязательно
          max_evals=50, # максимальное количество итераций
          trials=trials, # логирование результатов
          # rstate = np.random.default_rng(42) # для версии 0.2.7
          rstate=np.random.RandomState(42) # (random_state)# для версии 0.2.5, фиксируем для повторяемости результата
         )
print("Наилучшие значения гиперпараметров {}".format(best))

100%|███████████████████████████████████████████████| 50/50 [03:23<00:00,  4.06s/trial, best loss: -0.8005102732363687]
Наилучшие значения гиперпараметров {'max_depth': 30.0, 'min_samples_leaf': 5.0, 'n_estimators': 115.0}
Wall time: 3min 23s


In [28]:
# рассчитаем точность для тестовой выборки
model_HO_RF = ensemble.RandomForestClassifier(
    random_state=random_state, 
    n_estimators=int(best['n_estimators']),
    max_depth=int(best['max_depth']),
    min_samples_leaf=int(best['min_samples_leaf'])
)

model_HO_RF.fit(X_train, y_train)

y_test_pred = model_HO_RF.predict(X_test)
print('f1_score на тестовом наборе: {:.2f}'.format(metrics.f1_score(y_test, y_test_pred)))

f1_score на тестовом наборе: 0.82


In [29]:
# Записываем получившееся значение в словарь для дальнейшего сравнения
ranfor['RanFor_HO'] = round(metrics.f1_score(y_test, y_test_pred), 3)

# Метод Optuna

In [30]:
print("Версия Optuna: {}".format(optuna.__version__))

Версия Optuna: 3.0.2


### Логистическая регрессия, метод Optuna 

In [31]:
random_state=42
def optuna_lr(trial):
    penalty = trial.suggest_categorical('penalty', ['l1', 'l2'])
    solver = trial.suggest_categorical('solver', ['liblinear', 'saga'])
    C = trial.suggest_float(name='C', low=0.01, high=1.01, step = 0.05)
   
    ## Создаем модель
    model_Op_LR = linear_model.LogisticRegression(
                                    penalty=penalty,
                                    solver=solver,
                                    C=C,
                                    random_state=random_state
                              )
    ## Обучаем модель
    model_Op_LR.fit(X_train, y_train)
    score = cross_val_score(model_Op_LR, X_train, y_train, cv=5, scoring="f1", n_jobs=-1).mean()
    
    return score

In [32]:
%%time
study = optuna.create_study(study_name="LogisticRegression", direction="maximize")
study.optimize(optuna_lr, n_trials=50)

[I 2023-02-06 18:23:15,658] A new study created in memory with name: LogisticRegression
[I 2023-02-06 18:23:29,762] Trial 0 finished with value: 0.7743965200263728 and parameters: {'penalty': 'l1', 'solver': 'saga', 'C': 0.11}. Best is trial 0 with value: 0.7743965200263728.
[I 2023-02-06 18:23:46,204] Trial 1 finished with value: 0.7774415820631229 and parameters: {'penalty': 'l1', 'solver': 'saga', 'C': 0.6100000000000001}. Best is trial 1 with value: 0.7774415820631229.
[I 2023-02-06 18:23:57,155] Trial 2 finished with value: 0.763296420693192 and parameters: {'penalty': 'l2', 'solver': 'saga', 'C': 0.7100000000000001}. Best is trial 1 with value: 0.7774415820631229.
[I 2023-02-06 18:24:08,317] Trial 3 finished with value: 0.7644289379954936 and parameters: {'penalty': 'l2', 'solver': 'saga', 'C': 0.41000000000000003}. Best is trial 1 with value: 0.7774415820631229.
[I 2023-02-06 18:24:19,124] Trial 4 finished with value: 0.7644472151177506 and parameters: {'penalty': 'l2', 'solver'

[I 2023-02-06 18:26:57,429] Trial 39 finished with value: 0.7643926177712026 and parameters: {'penalty': 'l2', 'solver': 'saga', 'C': 0.46}. Best is trial 34 with value: 0.7788159787831656.
[I 2023-02-06 18:27:12,978] Trial 40 finished with value: 0.7753374538744907 and parameters: {'penalty': 'l1', 'solver': 'saga', 'C': 0.36000000000000004}. Best is trial 34 with value: 0.7788159787831656.
[I 2023-02-06 18:27:29,069] Trial 41 finished with value: 0.7788159787831656 and parameters: {'penalty': 'l1', 'solver': 'saga', 'C': 0.46}. Best is trial 34 with value: 0.7788159787831656.
[I 2023-02-06 18:27:45,072] Trial 42 finished with value: 0.7770546097143257 and parameters: {'penalty': 'l1', 'solver': 'saga', 'C': 0.51}. Best is trial 34 with value: 0.7788159787831656.
[I 2023-02-06 18:28:01,058] Trial 43 finished with value: 0.7788159787831656 and parameters: {'penalty': 'l1', 'solver': 'saga', 'C': 0.46}. Best is trial 34 with value: 0.7788159787831656.
[I 2023-02-06 18:28:16,200] Trial 4

Wall time: 6min 15s


In [33]:
# выводим наилучшие значения гиперпараметров
print("Наилучшие значения гиперпараметров {}".format(study.best_params))

Наилучшие значения гиперпараметров {'penalty': 'l1', 'solver': 'saga', 'C': 0.46}


In [34]:
model_Op_LR = linear_model.LogisticRegression(**study.best_params, random_state=random_state,)

model_Op_LR.fit(X_train, y_train)

y_test_pred = model_Op_LR.predict(X_test)
print('f1_score на тестовом наборе: {:.2f}'.format(metrics.f1_score(y_test, y_test_pred)))

f1_score на тестовом наборе: 0.79


In [35]:
# Записываем получившееся значение в словарь для дальнейшего сравнения
logreg['LogReg_Optuna'] = round(metrics.f1_score(y_test, y_test_pred), 3)

### Случайный лес, метод Optuna

In [36]:
random_state = 42
def optuna_rf(trial):
  # задаем пространства поиска гиперпараметров
  n_estimators = trial.suggest_int('n_estimators', 100, 400, 1)
  max_depth = trial.suggest_int('max_depth', 5, 30, 1)
  min_samples_leaf = trial.suggest_int('min_samples_leaf', 2, 12, 1)

  # создаем модель
  model_Op_rf = ensemble.RandomForestClassifier(n_estimators=n_estimators,
                                          max_depth=max_depth,
                                          min_samples_leaf=min_samples_leaf,
                                          random_state=random_state)
  # обучаем модель
  model_Op_rf.fit(X_train, y_train)
  score = cross_val_score(model_Op_rf, X_train, y_train, cv = 5, scoring = 'f1', n_jobs= -1).mean()
    
  return score

In [37]:
%%time
# cоздаем объект исследования
# можем напрямую указать, что нам необходимо максимизировать метрику direction="maximize"
study = optuna.create_study(study_name="RandomForestClassifier", direction="maximize")
# ищем лучшую комбинацию гиперпараметров n_trials раз
study.optimize(optuna_rf, n_trials=50)

[I 2023-02-06 18:30:21,056] A new study created in memory with name: RandomForestClassifier
[I 2023-02-06 18:30:29,678] Trial 0 finished with value: 0.7871897587442922 and parameters: {'n_estimators': 306, 'max_depth': 21, 'min_samples_leaf': 11}. Best is trial 0 with value: 0.7871897587442922.
[I 2023-02-06 18:30:37,657] Trial 1 finished with value: 0.7910445674446748 and parameters: {'n_estimators': 301, 'max_depth': 9, 'min_samples_leaf': 6}. Best is trial 1 with value: 0.7910445674446748.
[I 2023-02-06 18:30:43,394] Trial 2 finished with value: 0.7941998371835538 and parameters: {'n_estimators': 182, 'max_depth': 16, 'min_samples_leaf': 8}. Best is trial 2 with value: 0.7941998371835538.
[I 2023-02-06 18:30:48,427] Trial 3 finished with value: 0.7636537156330495 and parameters: {'n_estimators': 251, 'max_depth': 6, 'min_samples_leaf': 8}. Best is trial 2 with value: 0.7941998371835538.
[I 2023-02-06 18:30:52,845] Trial 4 finished with value: 0.7871385656938145 and parameters: {'n_e

[I 2023-02-06 18:35:50,043] Trial 38 finished with value: 0.8089988276411189 and parameters: {'n_estimators': 254, 'max_depth': 13, 'min_samples_leaf': 2}. Best is trial 24 with value: 0.8122918116434285.
[I 2023-02-06 18:36:00,554] Trial 39 finished with value: 0.8055979907464833 and parameters: {'n_estimators': 288, 'max_depth': 21, 'min_samples_leaf': 3}. Best is trial 24 with value: 0.8122918116434285.
[I 2023-02-06 18:36:06,029] Trial 40 finished with value: 0.7916269979232755 and parameters: {'n_estimators': 175, 'max_depth': 17, 'min_samples_leaf': 7}. Best is trial 24 with value: 0.8122918116434285.
[I 2023-02-06 18:36:16,545] Trial 41 finished with value: 0.8114895007480551 and parameters: {'n_estimators': 272, 'max_depth': 20, 'min_samples_leaf': 2}. Best is trial 24 with value: 0.8122918116434285.
[I 2023-02-06 18:36:26,741] Trial 42 finished with value: 0.8103795465112418 and parameters: {'n_estimators': 262, 'max_depth': 19, 'min_samples_leaf': 2}. Best is trial 24 with va

Wall time: 7min 10s


In [38]:
# выводим наилучшие значения гиперпараметров
print("Наилучшие значения гиперпараметров {}".format(study.best_params))

Наилучшие значения гиперпараметров {'n_estimators': 269, 'max_depth': 23, 'min_samples_leaf': 2}


In [39]:
# рассчитаем точность для тестовой выборки
model_Op_rf = ensemble.RandomForestClassifier(**study.best_params,random_state=random_state, )
model_Op_rf.fit(X_train, y_train)
y_test_pred = model_Op_rf.predict(X_test)
print('f1_score на тестовом наборе: {:.2f}'.format(metrics.f1_score(y_test, y_test_pred)))

f1_score на тестовом наборе: 0.82


In [40]:
# Записываем получившееся значение в словарь для дальнейшего сравнения
ranfor['RanFor_Optuna'] = round(metrics.f1_score(y_test, y_test_pred), 3)

In [41]:
# преобразуем словари  в датафреймы    
df_logreg = pd.DataFrame([logreg])
display(df_logreg)

df_ranfor = pd.DataFrame([ranfor])
display(df_ranfor)

,LogReg_init,LogReg_GridSeach,LogReg_RandSeach,LogReg_HO,LogReg_Optuna
0,0.782,0.794,0.794,0.796,0.794


,RanFor_init,RanFor_GridSeach,RanFor_RandSeach,RanFor_HO,RanFor_Optuna
0,0.817,0.819,0.817,0.817,0.824


### Некоторые выводы

1. Модель, использующая случайный лес, стабильно показывает более высокие результаты, чем модель, использующая логистическую регрессию при использовнии любого метода оптимизации
2. Наилучшие результаты для логистической регрессии показал метод HyperOpt (0.796), а для случайного леса - Optuna (0.825)